In [1]:
import cv2
import os
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import torch

from pathlib import Path
import sklearn
import skimage
from skimage import __version__ as skimage_version
from sklearn import __version__ as sklearn_version


path = "/content/"

print(f"Python version:           {os.sys.version.split()[0]}")
print(f"OpenCV version:           {cv2.__version__}")
print(f"NumPy version:            {np.__version__}")
print(f"Matplotlib version:       {matplotlib.__version__}")
print(f"PyTorch version:          {torch.__version__}")
print(f"scikit-image version:     {skimage_version}")
print(f"scikit-learn version:     {sklearn_version}")
print("Path: " + path)


Python version:           3.11.5
OpenCV version:           4.12.0
NumPy version:            2.2.6
Matplotlib version:       3.10.6
PyTorch version:          2.9.0+cpu
scikit-image version:     0.25.2
scikit-learn version:     1.7.2
Path: /content/


In [ ]:
!cd path
!kaggle competitions download -c ecse-415-video-analysis
!unzip ecse-415-video-analysis.zip -d .


In [ ]:
task1_dir = Path(path, "Object_Tracking", "Task1")
csv_path = Path(task1_dir, "gt", "gt.txt")
ground_truth = np.loadtxt("csv_path", delimiter=",",skiprows=1)






